## 파일 불러오기 및 형태소

In [2]:
import olefile
import zlib
import struct

In [3]:
# file_name = input('파일명: ')
file_name = '2020113128_권수아.hwp'

if file_name.split('.')[-1] == 'hwp':
    f = olefile.OleFileIO(file_name)
    encoded_text = f.openstream('\005HwpSummaryInformation').read()
    decoded_text = encoded_text.decode('utf-16', errors='ignore')
    print(decoded_text)

 ؂Ȁ悶ꊟ愐퐑듆`韀鶌Ā 悶ꊟ愐퐑듆`韀鶌　 ꤁ ฀ Ȁ 砀 ̀ 萀 Ѐ 退 ᐀ ꀀ Ԁ  ؀  ࠀ  ऀ Ё ఀ 栁 ഀ 琁 ଀ 老 ฀ 谁 ᔀ 鐁   鰁 ἀ Ā   ἀ Ā   ἀ Ѐ 峍㇁ ἀ ᠀ ㈀　㈀　䒱 㐀퓆 ㄀糇 Ⓠӈ ㄀　㨀㔀㜀㨀㌀　 ἀ Ā   ἀ Ā   ἀ ؀ 㠀㈀㄀　㔀 ἀ ⴀ 㠀Ⰰ 㔀Ⰰ 㠀Ⰰ ㄀㘀㜀㜀 圀䤀一㌀㈀䰀䔀圀椀渀搀漀眀猀开唀渀欀渀漀眀渀开嘀攀爀猀椀漀渀  䀀 灳젇혁䀀 탺噢눔혁䀀     ̀   ̀   Ā   Ā 


In [4]:
def get_hwp_text(filename):
    f = olefile.OleFileIO(filename)
    dirs = f.listdir()

    # HWP 파일 검증
    if ["FileHeader"] not in dirs or \
       ["\x05HwpSummaryInformation"] not in dirs:
        raise Exception("Not Valid HWP.")

    # 문서 포맷 압축 여부 확인
    header = f.openstream("FileHeader")
    header_data = header.read()
    is_compressed = (header_data[36] & 1) == 1

    # Body Sections 불러오기
    nums = []
    for d in dirs:
        if d[0] == "BodyText":
            nums.append(int(d[1][len("Section"):]))
    sections = ["BodyText/Section"+str(x) for x in sorted(nums)]

    # 전체 text 추출
    text = ""
    for section in sections:
        bodytext = f.openstream(section)
        data = bodytext.read()
        if is_compressed:
            unpacked_data = zlib.decompress(data, -15)
        else:
            unpacked_data = data
    
        # 각 Section 내 text 추출    
        section_text = ""
        i = 0
        size = len(unpacked_data)
        while i < size:
            header = struct.unpack_from("<I", unpacked_data, i)[0]
            rec_type = header & 0x3ff
            rec_len = (header >> 20) & 0xfff

            if rec_type in [67]:
                rec_data = unpacked_data[i+4:i+4+rec_len]
                section_text += rec_data.decode('utf-16', errors='ignore')
                section_text += "\n"

            i += 4 + rec_len

        text += section_text
        text += "\n"

    return text

In [5]:
get_hwp_text('2020113128_권수아.hwp')

'\x02捤獥\x00\x00\x00\x00\x02\x02汤捯\x00\x00\x00\x00\x022020년도 1학기 인공지능과 미래사회 개별과제(2차)\r\n2020.4.17\r\n생체 공학의 발전\r\n권수아 (2020113128) \r\n동국대학교 글로벌무역학과\r\nᒦ\x00생체 공학 디자이너인 휴 허 박사는 1982년 산악 등반 사고에서 등산으로 인한 조직 손상으로 두 다리를 절단하는 아픔을 겼었습니다. 휴 허 박사는 본인 몸이 무너졌다고 생각하지 않았으며 사람은 절대 좌절하지 않는다고 외쳤습니다. 하지만 생체 공학적 기술은 항상 부족하다고 느꼈습니다. 휴 허 박사는 다른 사람들의 장애를 제거하는 발전된 기술을 가져야겠다고 결심하여 이러한 기폭제를 통해 바위와 얼음벽을 수직으로 오르는 특화된 수족을 개발하기 시작했습니다. 또한 인공신체가 쉽게 변할 수 있어야 하고 어떤 형태든 잘 수행할 수 있을 정도로 만능이어서 생물학적 능력을 넘어설 수 있는 구조물을 만들어 내야 한다고 생각하여 본인 스스로 높이를 조절하며 키를 크게 할 수 있도록 개발하고 자기 자신이 불쾌하거나 불편하면 스스로 조절할 수 있게 개발하며 본인 몸에 맞춰 발전해나갔습니다. 또한 기술 혁신을 통해 주인공 운동 능력을 더 강하게 하여 장애를 없애주었고 새로운 등산 기술을 배우며 희망을 가져다 줄 수 있었습니다. 주인공은 항상 기술이 엄청나게 발달하여 세상에서 장애를 없앨 수 있고 마비 환자가 외골격 신체를 이용해 걸을 수 있는 세상을 꿈꾸었습니다. 하지만 아직까지 슬프게 세상에는 많은 장애가 만연해 있습니다. 이러한 문제를 해결하기 위해 박사는 MIT 미디어 랩 연구실에 극단적 생체 공학 센터를 설립했습니다. 이 센터의 임무는 기초 과학과 기술적 능력을 증진하여 뇌와 신체의 모든 장애에 대한 생체 메카트로닉스와 인체의 재생성 회복을 가능하도록 하는 곳입니다. 이곳에서는 정상적인 사람들이 서거나 걷고 뛰는지에 대한 연구를 통하여 근육이 하는 일, 척수에 근육이 어떻게 이용되는지 연구하여 무릎,

## doc file

In [6]:
# pip install docx2txt

Note: you may need to restart the kernel to use updated packages.


In [1]:
import docx2txt

In [2]:
text = docx2txt.process('2019113108_김혜영.docx')
print(text)

우리생활 속의 생체공학



김혜영(2019113108)

동국대학교 글로벌 무역학과



- (서론1) 동영상의 주요 내용이나 흥미롭거나 관심 있었던 내용 소개

: 이 동영상은 생체공학에 대해 다룬 영상이다. 연설자 휴 허는 산악 등반사고로 동상을 입어 두 다리를 절단한 것을 계기로 사람들의 장애를 제거할 수 있는 기술이 필요하다고 생각하였다. 그는 생체공학이 더 좋아져 언젠가, 미래에는 팔다리 세 군데가 없는 부상도 완전히 회복시킬 수 있어야 한다고 생각하였고 MIT 미디어 랩에 극단적 생체공학 센터를 설립하였다. 그는 여러 연구를 통해 연동이 조직 전체의 변화에 따라 일어나야 하는 최적의 시점을 찾아 생체 공학적 의족을 만들어냈다. 또 그는 미래에는 옷, 신발, 보조기, 인공기관들이 더 이상 디자인되거나 자동화과정으로 생산되는 것이 아닌 자료를 이용한 수치적 체계를 통해 생산될 것이라고 말하였다. 의족 사용하는 두 가지 사례를 더 말한 후 보스톤 테러 사건으로 왼쪽 다리를 잃은 무도회 댄서 아드리안의 이야기를 하였다. 연설자는 그녀에게 의족을 만들어주어 그녀의 삶인 춤으로 되돌아갈 수 있도록 해주자는 생각에 200일 이상 춤에 대해 연구를 하고 아드리안에게 의족을 통해 무대에 설 수 있도록 해주었다. 영상은 아드리안의 춤으로 마무리가 되었다. 생체공학이 미래에 어떤 변화를 불러올지, 그 것이 인간에게 얼마나 도움이 되는지를 설명하는 것이 주요 내용이다.

나는 그가 두 다리를 절단하였음에도 좌절하지 않고, 오히려 다른 사람의 장애를 제거하는 발전된 기술을 가져야겠다는 미래지향적인 생각을 한 것이정말 대단하다고 생각했다. 만약 나에게 이런 일이 생겼더라면 앞으로 살아갈 미래에 대한 걱정과 좌절이 먼저 앞섰을것 같은데 말이다. 그가 위와 같은 생각을 하였기에 의족을 개발하고 생체공학 센터를 설립하고 연설까지 할 수 있다는 것이라 생각이 되었고 걱정 가득한 부정적인 생각을 먼저 하게 되는 나를 돌아보게 되었던 것 같다. 흥미로웠던 부분은 사례로 나왔던 분들의

## 의미망분석

In [3]:
import pandas as pd
import MeCab

In [18]:
from eunjeon import Mecab

AttributeError: 'NoneType' object has no attribute 'DictionaryInfo_filename_get'

In [4]:
a = MeCab.Tagger('-d /usr/local/lib/mecab/dic/mecab-ko-dic')
ret = a.parse(text)
ret

'우리\tNP,*,F,우리,*,*,*,*\n생활\tNNG,행위,T,생활,*,*,*,*\n속\tNNG,*,T,속,*,*,*,*\n의\tJKG,*,F,의,*,*,*,*\n생체\tNNG,*,F,생체,*,*,*,*\n공학\tNNG,*,T,공학,*,*,*,*\n김혜영\tNNP,인명,T,김혜영,*,*,*,*\n(\tSSO,*,*,*,*,*,*,*\n2019113108\tSN,*,*,*,*,*,*,*\n)\tSSC,*,*,*,*,*,*,*\n동국\tNNG,*,T,동국,*,*,*,*\n대학교\tNNG,*,F,대학교,Compound,*,*,대/NNG/*+학교/NNG/*\n글로벌\tNNG,*,T,글로벌,*,*,*,*\n무역\tNNG,행위,T,무역,*,*,*,*\n학과\tNNG,*,F,학과,*,*,*,*\n-\tSY,*,*,*,*,*,*,*\n(\tSSO,*,*,*,*,*,*,*\n서론\tNNG,*,T,서론,*,*,*,*\n1\tSN,*,*,*,*,*,*,*\n)\tSSC,*,*,*,*,*,*,*\n동영상\tNNG,*,T,동영상,Compound,*,*,동/NNG/*+영상/NNG/*\n의\tJKG,*,F,의,*,*,*,*\n주요\tNNG,*,F,주요,*,*,*,*\n내용\tNNG,*,T,내용,*,*,*,*\n이나\tJC,*,F,이나,*,*,*,*\n흥미\tNNG,정적사태,F,흥미,*,*,*,*\n롭\tXSA,*,T,롭,*,*,*,*\n거나\tEC,*,F,거나,*,*,*,*\n관심\tNNG,행위,T,관심,*,*,*,*\n있\tVA,*,T,있,*,*,*,*\n었\tEP,*,T,었,*,*,*,*\n던\tETM,*,T,던,*,*,*,*\n내용\tNNG,*,T,내용,*,*,*,*\n소개\tNNG,행위,F,소개,*,*,*,*\n:\tSC,*,*,*,*,*,*,*\n이\tMM,~명사,F,이,*,*,*,*\n동영상\tNNG,*,T,동영상,Compound,*,*,동/NNG/*+영상/NNG/*\n은\tJX,*,T,은,*,*,*,*\n생체\tNNG,*,F,생체,*,*,*,*\n공학\tNNG,*,T,공

In [5]:
m = MeCab.Tagger()
te = m.parse(text)
print(te)

우리	NP,*,F,우리,*,*,*,*
생활	NNG,행위,T,생활,*,*,*,*
속	NNG,*,T,속,*,*,*,*
의	JKG,*,F,의,*,*,*,*
생체	NNG,*,F,생체,*,*,*,*
공학	NNG,*,T,공학,*,*,*,*
김혜영	NNP,인명,T,김혜영,*,*,*,*
(	SSO,*,*,*,*,*,*,*
2019113108	SN,*,*,*,*,*,*,*
)	SSC,*,*,*,*,*,*,*
동국	NNG,*,T,동국,*,*,*,*
대학교	NNG,*,F,대학교,Compound,*,*,대/NNG/*+학교/NNG/*
글로벌	NNG,*,T,글로벌,*,*,*,*
무역	NNG,행위,T,무역,*,*,*,*
학과	NNG,*,F,학과,*,*,*,*
-	SY,*,*,*,*,*,*,*
(	SSO,*,*,*,*,*,*,*
서론	NNG,*,T,서론,*,*,*,*
1	SN,*,*,*,*,*,*,*
)	SSC,*,*,*,*,*,*,*
동영상	NNG,*,T,동영상,Compound,*,*,동/NNG/*+영상/NNG/*
의	JKG,*,F,의,*,*,*,*
주요	NNG,*,F,주요,*,*,*,*
내용	NNG,*,T,내용,*,*,*,*
이나	JC,*,F,이나,*,*,*,*
흥미	NNG,정적사태,F,흥미,*,*,*,*
롭	XSA,*,T,롭,*,*,*,*
거나	EC,*,F,거나,*,*,*,*
관심	NNG,행위,T,관심,*,*,*,*
있	VA,*,T,있,*,*,*,*
었	EP,*,T,었,*,*,*,*
던	ETM,*,T,던,*,*,*,*
내용	NNG,*,T,내용,*,*,*,*
소개	NNG,행위,F,소개,*,*,*,*
:	SC,*,*,*,*,*,*,*
이	MM,~명사,F,이,*,*,*,*
동영상	NNG,*,T,동영상,Compound,*,*,동/NNG/*+영상/NNG/*
은	JX,*,T,은,*,*,*,*
생체	NNG,*,F,생체,*,*,*,*
공학	NNG,*,T,공학,*,*,*,*
에	JKB,*,F,에,*,*,*,*
대해	VV+EC,*,F,대해,Inflect,VV,EC,대하/VV/*+아/EC/*
다룬	VV

In [6]:
df = pd.read_excel('stopwords.xlsx')
stopwords = list(df['불용어'])
stopwords

['아',
 '휴',
 '아이구',
 '아이쿠',
 '아이고',
 '어',
 '나',
 '우리',
 '저희',
 '따라',
 '의해',
 '을',
 '를',
 '에',
 '의',
 '가',
 '으로',
 '로',
 '에게',
 '뿐이다',
 '의거하여',
 '근거하여',
 '입각하여',
 '기준으로',
 '예하면',
 '예를 들면',
 '예를 들자면',
 '저',
 '소인',
 '소생',
 '저희',
 '지말고',
 '하지마',
 '하지마라',
 '다른',
 '물론',
 '또한',
 '그리고',
 '비길수 없다',
 '해서는 안된다',
 '뿐만 아니라',
 '만이 아니다',
 '만은 아니다',
 '막론하고',
 '관계없이',
 '그치지 않다',
 '그러나',
 '그런데',
 '하지만',
 '든간에',
 '논하지 않다',
 '따지지 않다',
 '설사',
 '비록',
 '더라도',
 '아니면',
 '만 못하다',
 '하는 편이 낫다',
 '불문하고',
 '향하여',
 '향해서',
 '향하다',
 '쪽으로',
 '틈타',
 '이용하여',
 '타다',
 '오르다',
 '제외하고',
 '이 외에',
 '이 밖에',
 '하여야',
 '비로소',
 '한다면 몰라도',
 '외에도',
 '이곳',
 '여기',
 '부터',
 '기점으로',
 '따라서',
 '할 생각이다',
 '하려고하다',
 '이리하여',
 '그리하여',
 '그렇게 함으로써',
 '하지만',
 '일때',
 '할때',
 '앞에서',
 '중에서',
 '보는데서',
 '으로써',
 '로써',
 '까지',
 '해야한다',
 '일것이다',
 '반드시',
 '할줄알다',
 '할수있다',
 '할수있어',
 '임에 틀림없다',
 '한다면',
 '등',
 '등등',
 '제',
 '겨우',
 '단지',
 '다만',
 '할뿐',
 '딩동',
 '댕그',
 '대해서',
 '대하여',
 '대하면',
 '훨씬',
 '얼마나',
 '얼마만큼',
 '얼마큼',
 '남짓',
 '여',
 '얼마간',
 '약간',
 '다소',
 '좀',
 '조

In [17]:
print(te)

우리	NP,*,F,우리,*,*,*,*
생활	NNG,행위,T,생활,*,*,*,*
속	NNG,*,T,속,*,*,*,*
의	JKG,*,F,의,*,*,*,*
생체	NNG,*,F,생체,*,*,*,*
공학	NNG,*,T,공학,*,*,*,*
김혜영	NNP,인명,T,김혜영,*,*,*,*
(	SSO,*,*,*,*,*,*,*
2019113108	SN,*,*,*,*,*,*,*
)	SSC,*,*,*,*,*,*,*
동국	NNG,*,T,동국,*,*,*,*
대학교	NNG,*,F,대학교,Compound,*,*,대/NNG/*+학교/NNG/*
글로벌	NNG,*,T,글로벌,*,*,*,*
무역	NNG,행위,T,무역,*,*,*,*
학과	NNG,*,F,학과,*,*,*,*
-	SY,*,*,*,*,*,*,*
(	SSO,*,*,*,*,*,*,*
서론	NNG,*,T,서론,*,*,*,*
1	SN,*,*,*,*,*,*,*
)	SSC,*,*,*,*,*,*,*
동영상	NNG,*,T,동영상,Compound,*,*,동/NNG/*+영상/NNG/*
의	JKG,*,F,의,*,*,*,*
주요	NNG,*,F,주요,*,*,*,*
내용	NNG,*,T,내용,*,*,*,*
이나	JC,*,F,이나,*,*,*,*
흥미	NNG,정적사태,F,흥미,*,*,*,*
롭	XSA,*,T,롭,*,*,*,*
거나	EC,*,F,거나,*,*,*,*
관심	NNG,행위,T,관심,*,*,*,*
있	VA,*,T,있,*,*,*,*
었	EP,*,T,었,*,*,*,*
던	ETM,*,T,던,*,*,*,*
내용	NNG,*,T,내용,*,*,*,*
소개	NNG,행위,F,소개,*,*,*,*
:	SC,*,*,*,*,*,*,*
이	MM,~명사,F,이,*,*,*,*
동영상	NNG,*,T,동영상,Compound,*,*,동/NNG/*+영상/NNG/*
은	JX,*,T,은,*,*,*,*
생체	NNG,*,F,생체,*,*,*,*
공학	NNG,*,T,공학,*,*,*,*
에	JKB,*,F,에,*,*,*,*
대해	VV+EC,*,F,대해,Inflect,VV,EC,대하/VV/*+아/EC/*
다룬	VV